In [15]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices, _5_perf_ranks
from yf_utils import _6_grp_tuples_sort_sum

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [16]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [17]:
n_samples = 4  # number of tuples to create for iloc start_train:end_train:end_eval
# n_samples = 100  # number of tuples to create for iloc start_train:end_train:end_eval
days_lookbacks = [30, 120, 60]  # number of lookback days for training
days_eval = 10  # number of days to evaluate effectiveness of the training
# set_top_syms = 5  # number of the most-common symbols from days_lookbacks' performance rankings
set_top_syms = 10  # number of the most-common symbols from days_lookbacks' performance rankings to keep
idx_eval_start = 0  #  start index of set_top_syms for evaluation
idx_eval_end = 2  #  end index of set_top_syms for evaluation


In [18]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(672, 792, 802), (282, 402, 412), (176, 296, 306), (388, 508, 518)]
sets_lookback_slices:
[[(762, 792, 802), (732, 792, 802), (672, 792, 802)], [(372, 402, 412), (342, 402, 412), (282, 402, 412)], [(266, 296, 306), (236, 296, 306), (176, 296, 306)], [(478, 508, 518), (448, 508, 518), (388, 508, 518)]]


In [19]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval

    if verbose:
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days lookback: {lookback}')
      print(f'days eval:     {eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=set_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:set_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[idx_eval_start:idx_eval_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {set_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {set_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {set_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

lb_slices:     [(762, 792, 802), (732, 792, 802), (672, 792, 802)]
lb_slice:      (762, 792, 802)
days lookback: 30
days eval:     10
start_train:   762
end_train:     792
perf_ranks: {'period-30': {'r_CAGR/UI': array(['EQT', 'NVAX', 'REGN', 'KR', 'TDOC', 'CLX', 'VIRT', 'UNFI', 'GILD',
       'EVBG'], dtype=object), 'r_CAGR/retnStd': array(['NVAX', 'EQT', 'VIRT', 'REGN', 'KR', 'TDOC', 'UNFI', 'CLX', 'SPTN',
       'GILD'], dtype=object), 'r_retnStd/UI': array(['CLX', 'EQT', 'TPVG', 'EVBG', 'CCOI', 'FLO', 'REGN', 'CTRA', 'GIS',
       'CALM'], dtype=object)}}
most_common_syms: [('EQT', 3), ('REGN', 3), ('CLX', 3), ('NVAX', 2), ('KR', 2), ('TDOC', 2), ('VIRT', 2), ('UNFI', 2), ('GILD', 2), ('EVBG', 2), ('SPTN', 1), ('TPVG', 1), ('CCOI', 1), ('FLO', 1), ('CTRA', 1), ('GIS', 1), ('CALM', 1)]
+++ finish lookback slice 30 +++

lb_slices:     [(762, 792, 802), (732, 792, 802), (672, 792, 802)]
lb_slice:      (732, 792, 802)
days lookback: 60
days eval:     10
start_train:   732
end_train:    

In [20]:
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
for item in z_grp_top_set_syms:
  print(item)
print('')

z_grp_top_set_syms:
((672, 792, 802), ['CLX', 'EVBG'])
((282, 402, 412), ['MOH', 'TNDM'])
((176, 296, 306), ['ENPH', 'TNDM'])
((388, 508, 518), ['AXSM', 'SBS'])



In [21]:
from yf_utils import _7_perf_eval
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
# z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms[26:29])

for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms):
# for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms[26:29]):

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  _date_0 = df_train.index[_lookback_slice[0]].strftime('%Y-%m-%d')
  _date_1 = df_train.index[_lookback_slice[1]].strftime('%Y-%m-%d')
  _date_2 = df_train.index[_lookback_slice[2]].strftime('%Y-%m-%d')
  print(f'max lookback dates: {_date_0}, {_date_1}, {_date_2}')  
  print(f'top_set_syms: {_top_set_syms}\n')

  start_eval = _lookback_slice[1]
  end_eval = _lookback_slice[2]
  df_eval = df_train[start_eval:end_eval][_top_set_syms]

  if verbose:
    print(f'start_eval: {start_eval}')
    print(f'end_eval:   {end_eval}')  
    print(f'\ndf_eval:\n{df_eval}\n')


  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
  print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_retnStd_d_UI[1]:>13,.3f}, {grp_retnStd_d_UI[2]:>13,.3f}')
  print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
  print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>13,.3f}, {grp_CAGR_d_UI[1]:>13,.3f}, {grp_CAGR_d_UI[2]:>13,.3f}')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]
  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_SPY)
  # print(f'\n_sym_idx: {_sym_idx}')
  # print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_retnStd_d_UI[1]:>13,.3f}, {grp_retnStd_d_UI[2]:>13,.3f}')
  # print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
  # print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>13,.3f}, {grp_CAGR_d_UI[1]:>13,.3f}, {grp_CAGR_d_UI[2]:>13,.3f}')
  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_UI[0]:>13,.3f}')
  print('='*50, '\n')  

z_grp_top_set_syms:
1 of 4 max_lookback_slice
max_lookback_slice: (672, 792, 802)
max lookback dates: 2019-09-27, 2020-03-20, 2020-04-03
top_set_syms: ['CLX', 'EVBG']

start_eval: 792
end_eval:   802

df_eval:
                   CLX        EVBG
Date                              
2020-03-20  169.307785  100.269997
2020-03-23  162.378937  115.970001
2020-03-24  160.641983  122.589996
2020-03-25  158.103302  110.669998
2020-03-26  165.213486  112.699997
2020-03-27  165.289841  108.949997
2020-03-30  166.520966  112.690002
2020-03-31  165.347076  106.360001
2020-04-01  166.692749  103.010002
2020-04-02  171.502853  103.279999

grp(retnStd/UI):   mean, std, mean/std:         0.745,         0.022,        33.429
grp(CAGR/retnStd): mean, std, mean/std:        15.128,         0.214,        70.599
grp(CAGR/UI):      mean, std, mean/std:        11.276,         0.497,        22.692

SPY: retnStd/UI, CAGR/retnStd, CAGR/UI:         1.731,       228.233,       395.018

2 of 4 max_lookback_slice
max_l